In [1]:
import sys
import os

# Add parent directory to system path
sys.path.append(os.path.abspath(".."))
import connect_db

from importlib import reload

reload(connect_db)
ConnectDB = connect_db.ConnectDB

In [9]:
db = ConnectDB(db_path='../app_storage/metadata/sqlite-poc.db')
cursor = db.connection.cursor()
cursor.execute("SELECT * FROM fts_chunks").fetchall()


[('competent generalists. We would like to move towards more\ngeneral systems which can perform many tasks – eventually\nwithout the need to manually create and label a training\ndataset for each one. Natural language processing tasks, such as ques-\ntion answering, machine translation, reading com-\nprehension, and summarization, are typically\napproached with supervised learning on task-\nspeciﬁc datasets. We demonstrate that language\nmodels begin to learn these tasks without any ex-\nplicit supervision when trained on a new dataset\nof millions of webpages called WebText. When\nconditioned on a document plus questions, the an-\nswers generated by the language model reach 55\nF1 on the CoQA dataset - matching or exceeding\nthe performance of 3 out of 4 baseline systems\nwithout using the 127,000+ training examples. The capacity of the language model is essential\nto the success of zero-shot task transfer and in-\ncreasing it improves performance in a log-linear\nfashion across tasks

In [12]:
import sqlite3
import pandas as pd
import sqlite_vec



sqlite_connection = sqlite3.connect('../app_storage/metadata/sqlite-poc.db')
sqlite_connection.enable_load_extension(True)
sqlite_vec.load(sqlite_connection)
sqlite_connection.enable_load_extension(False)


pd.read_sql_query("SELECT * FROM pdf_metadata;", sqlite_connection).head()


,id,file_name,title,author,subject,keywords,creation_date
0,1,None,,,,,2024-04-10
1,2,None,,,,,2024-04-10
2,3,None,,,,,2024-04-10
3,4,None,Language Models are Unsupervised Multitask Lea...,"Alec Radford*, Jeffrey Wu*, Rewon Child, David...",Proceedings of the International Conference on...,"Machine Learning, ICML",2019-02-14
4,5,None,,,,,2020-07-24


In [ ]:
import sqlite3
import sqlite_vec
from typing import List
import struct




k=10
rrf_k=60
weight_fts=1.0
weight_vec=1.0

                
query = f"""
    -- extract list of possible chunks from documents
    with list_of_possible_chunks as (
        select chunk_id, 
            document_id 
        from chunks
        where document_id in (:documents)
    ),
    
    -- the sqlite-vec KNN vector search results
    vec_matches as (
    select
        chunk_id,
        row_number() over (order by distance) as rank_number,
        distance
    from chunk_embeddings
    where
        embedding match :embedded_query
        and chunk_id in (select chunk_id from list_of_possible_chunks)
        and k = :k
    ),
    
    -- the FTS5 search results
    fts_matches as (
    select
        rowid,
        row_number() over (order by rank) as rank_number,
        rank as score
    from fts_chunks
    where 
        text match :query
        and rowid in (select chunk_id from list_of_possible_chunks)
    limit :k
    ),
    
    -- combine FTS5 + vector search results with RRF
    final as (
    select
        chunks.chunk_id,
        chunks.text,
        vec_matches.rank_number as vec_rank,
        fts_matches.rank_number as fts_rank,
        -- RRF algorithm
        (
        coalesce(1.0 / (:rrf_k + fts_matches.rank_number), 0.0) * :weight_fts +
        coalesce(1.0 / (:rrf_k + vec_matches.rank_number), 0.0) * :weight_vec
        ) as combined_rank,
        vec_matches.distance as vec_distance,
        fts_matches.score as fts_score
    from fts_matches
    full outer join vec_matches on vec_matches.chunk_id = fts_matches.rowid
    join chunks on chunks.chunk_id = coalesce(fts_matches.rowid, vec_matches.chunk_id)
    order by combined_rank desc
    )
    
    select * from final
"""
document_string = ', '.join(str(element) for element in documents)

cursor.execute(query, 
                    {
                    'documents': document_string,
                    'embedded_query': serialize_f32(embedded_query),
                    'query': query,
                    'k': k,
                    'rrf_k': rrf_k,
                    'weight_fts': weight_fts,
                    'weight_vec': weight_vec
                    }
                    )

# Fetch and return results
results = cursor.fetchall()
columns = [description[0] for description in cursor.description]
return [dict(zip(columns, row)) for row in results]


In [46]:
from typing import List
import struct

documents = [1, 2]
placeholders = ', '.join('?' for _ in documents)
k=10
rrf_k=60
weight_fts=1.0
weight_vec=1.0
query = "hello"
embedded_query = [1.0, 2.0, 3.0, 0.0]

def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)

cursor.execute("DROP TABLE IF EXISTS temp_list_of_possible_chunks;")

create_temp_table_query = f"""
    CREATE TEMP TABLE temp_list_of_possible_chunks AS
    WITH list_of_possible_chunks AS (
        SELECT chunk_id, document_id 
        FROM chunks
        WHERE document_id IN ({placeholders})
    )
    SELECT * FROM list_of_possible_chunks;
"""

cursor.execute(create_temp_table_query, documents)

query = """
    with vec_matches as (
    select
        chunk_id,
        row_number() over (order by distance) as rank_number,
        distance
    from chunk_embeddings
    where
        embedding match :embedded_query
        and chunk_id in (select chunk_id from temp_list_of_possible_chunks)
        and k = :k
    )
    Select * from vec_matches
    """
    
cursor.execute(query, 
                    {
                    'embedded_query': serialize_f32(embedded_query),
                    'k': k
                    }
                    ).fetchall()

[(8, 1, 2.4220290184020996),
 (59, 2, 2.4528276920318604),
 (70, 3, 2.476188898086548),
 (13, 4, 2.5906496047973633),
 (27, 5, 2.601496458053589),
 (79, 6, 2.625840187072754),
 (29, 7, 2.639091730117798),
 (62, 8, 2.6459600925445557),
 (5, 9, 2.6789605617523193),
 (6, 10, 2.6836345195770264)]

In [44]:
from typing import List
import struct

documents = [1, 2]
placeholders = ', '.join('?' for _ in documents)
k=10
rrf_k=60
weight_fts=1.0
weight_vec=1.0
text = "transformer"
embedded_query = [1.0, 2.0, 3.0, 0.0]

def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)

cursor.execute("DROP TABLE IF EXISTS temp_list_of_possible_chunks;")

create_temp_table_query = f"""
    CREATE TEMP TABLE temp_list_of_possible_chunks AS
    WITH list_of_possible_chunks AS (
        SELECT chunk_id, document_id 
        FROM chunks
        WHERE document_id IN ({placeholders})
    )
    SELECT * FROM list_of_possible_chunks;
"""

cursor.execute(create_temp_table_query, documents).fetchall()

query = """
    with fts_matches as (
    select
        rowid,
        row_number() over (order by rank) as rank_number,
        rank as score
    from fts_chunks
    where 
        text match :query
        and rowid in (select chunk_id from temp_list_of_possible_chunks)
    limit :k
    )
    select * from fts_matches
    """
    
cursor.execute(query, 
                    {
                    'query': text,
                    'k': k
                    }
                    ).fetchall()

[(33, 1, -2.1544734363919624),
 (1, 2, -2.0698722944831123),
 (4, 3, -2.0314641061490772),
 (14, 4, -1.9805777538298508)]

In [42]:
query

"\n    .param set :query 'abortion ban'\n    .param set :k 10\n\n    with fts_matches as (\n    select\n        rowid,\n        row_number() over (order by rank) as rank_number,\n        rank as score\n    from fts_chunks\n    where \n        text match :query\n        and rowid in (select chunk_id from temp_list_of_possible_chunks)\n    limit :k\n    )\n    select * from fts_matches\n    "

In [38]:
pd.read_sql_query("SELECT * FROM fts_chunks;", db.connection).head()

,text
0,competent generalists. We would like to move t...
1,"Multitask learning (Caruana, 1997) is a promis..."
2,"*, **Equal contribution\n1OpenAI, San Francisc..."
3,Machine learning systems now excel (in expecta...
4,These methods still require supervised trainin...
